In [9]:
%matplotlib inline
import nibabel as nib
import numpy as np
from scipy import optimize
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import subprocess


In [ ]:
import sys
sys.path.append('/Users/evaalonsoortiz/Documents/python/susceptibility-to-fieldmap-fft/')
from functions import compute_fieldmap

In [6]:
# Select the path to the folder so that it can always address data folder correctly
home_path = "C:/Users/User/msc_project/sus-optimization"

In [7]:
# Above method uses the "main" simulation, so we need to load the latest
!git clone https://github.com/shimming-toolbox/susceptibility-to-fieldmap-fft.git
%cd susceptibility-to-fieldmap-fft
!git checkout mb/custom_pad
%pip install .
%cd {home_path}

c:\Users\User\msc_project\sus-optimization\susceptibility-to-fieldmap-fft


fatal: destination path 'susceptibility-to-fieldmap-fft' already exists and is not an empty directory.
c:\Users\User\miniconda3\envs\mrsim\Lib\site-packages\IPython\core\magics\osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


Your branch is up to date with 'origin/mb/custom_pad'.


Already on 'mb/custom_pad'


Processing c:\users\user\msc_project\sus-optimization\susceptibility-to-fieldmap-fft
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for susceptibility-to-fieldmap-fft: filename=susceptibility_to_fieldmap_fft-0.0.0-py3-none-any.whl size=23545 sha256=56eedc2450496aeec029d98fca3e354a87c2671ddee2371874083cbaa24d03bd
  Stored in directory: c:\users\user\appdata\local\pip\cache\wheels\17\ea\51\22427b7b5d1aced006c3db69245d8a8f88effcb4349a030d61
Successfully built susceptibility-to-fieldmap-fft
  Attempting uninstall: susceptibility-to-fieldmap-fft
    Found existing installation: susceptibility-to-fieldmap-fft 0.0.0
    Uninstalling susceptibility-to-fieldmap-fft-0.0.0:
      Successfully uninstalled susceptibility-to-fieldmap-fft-0.0.0
Note: you may need to restart the kernel to use updated packages.
C:\Users\User\msc_project\sus-optimization


In [20]:
meas_fmap_path = "../data/fm_Hz.nii.gz"
sim_chi_map_path = "../data/chimap_crop_custom6.nii.gz"
ROIs_map_path = "../data/mask_crop_gre_fov_body_plus_trachea.nii.gz" # Segmentations

In [2]:
meas_fmap_path2 = "data/demod_ISMRM_sim_fm_Hz.nii.gz"
sim_chi_map_path2 = "data/db0_030_wb_chimap.nii.gz"
segmentations = "data/db0_030_segmentations.nii.gz"
sc_path = "data/t1w_wholebody_sc.nii.gz"

In [13]:
# We need to reate the custom command that creates the fieldmap
def chi_to_fm_ppm(input_file, output_file):

    #Using a custom command for dB0 simulation

    command = [
    "compute_fieldmap",
    "-i", input_file,
    "-o", output_file,
    "-b", "50",
    "-m", "b0SimISMRM"
    ]

    # Run the command and capture output
    result = subprocess.run(command, capture_output=True, text=True)

    # We need to return the data from the fieldmap created

    
    # Check for errors
    if result.returncode != 0:
        print("Error occurred:", result.stderr)

    else:
        print("Fieldmap simulated correctly", result.stdout)
        sim_fm_img = nib.load(output_file)
        sim_fm_data = sim_fm_img.get_fdata()
        return sim_fm_data

In [15]:
def f(x):

    # load simulated susceptibility map in ppm
    chi_img = nib.load(sim_chi_map_path2)
    chi_data = chi_img.get_fdata()
    
    # load segmentation labels map
    # Because the spinal cord segmentation and the whole body segmentation have same dimensions this must work
    ROIs_img = nib.load(segmentations)
    ROIs_data = ROIs_img.get_fdata()

    sc_seg_img = nib.load(sc_path)
    sc_seg_data = sc_seg_img.get_fdata()
    # find indicies where voxels correspond to the spinal cord
    ind_sc = np.where(sc_seg_data == 1)
    # find indicies that have label = 7 or 8, these voxels should correspond to the trachea (t) and lungs (l); ind_tl
    ind_tl = np.where((ROIs_data == 7) | (ROIs_data == 8))
    
    # load simulated field map in Hz
    b0meas_img = nib.load(meas_fmap_path2)
    b0meas_data = b0meas_img.get_fdata()

    # assign variable "x" to indicies corresponding to trachea/lungs
    chi_data[ind_tl] = x

    # compute the field distribution in ppm for the chi_data distribution using the compute_bz function from the susceptibility-to-fieldmap-fft package
    #z =  compute_fieldmap.compute_bz(chi_data)
    # convert field distribution from ppm to Hz (assuming 3T)
    z = chi_to_fm_ppm(sim_chi_map_path2,"output/testing_fm.nii.gz")
    z = z*3*42.58 # deltaB0 [Hz] = deltaB0 [ppm] * B0 [T] * 1e-6 * 42.58*e6 [Hz/T]

    return (np.linalg.norm(b0meas_data[ind_sc] - z[ind_sc]))

In [ ]:
min_chi_tl = optimize.fmin(f, 0.3)
min_chi_tl[0]

Fieldmap simulated correctly Start
Susceptibility distribution loaded
Fieldmap simulated
Saving to NIfTI format
End. Runtime: 26.23 seconds

Fieldmap simulated correctly Start
Susceptibility distribution loaded
Fieldmap simulated
Saving to NIfTI format
End. Runtime: 24.50 seconds

Fieldmap simulated correctly Start
Susceptibility distribution loaded
Fieldmap simulated
Saving to NIfTI format
End. Runtime: 23.89 seconds

Fieldmap simulated correctly Start
Susceptibility distribution loaded
Fieldmap simulated
Saving to NIfTI format
End. Runtime: 23.62 seconds

Fieldmap simulated correctly Start
Susceptibility distribution loaded
Fieldmap simulated
Saving to NIfTI format
End. Runtime: 23.56 seconds

Fieldmap simulated correctly Start
Susceptibility distribution loaded
Fieldmap simulated
Saving to NIfTI format
End. Runtime: 23.98 seconds

Fieldmap simulated correctly Start
Susceptibility distribution loaded
Fieldmap simulated
Saving to NIfTI format
End. Runtime: 26.21 seconds

Fieldmap simu

## Change to f </br>
For the new chimaps, there is no spinal cord added but we have the spina cord segmentation

In [ ]:
def f2(x):

    # load simulated susceptibility map in ppm
    chi_img = nib.load(sim_chi_map_path)
    chi_data = chi_img.get_fdata()
    
    # load segmentation labels map
    ROIs_img = nib.load(ROIs_map_path)
    ROIs_data = ROIs_img.get_fdata()
    # find indicies that have label = 256, these voxels should correspond to the spinal cord
    ind_sc = np.where(sc_path == 256)
    # find indicies that have label = 7 or 8, these voxels should correspond to the trachea (t) and lungs (l); ind_tl
    ind_tl = np.where((ROIs_data == 7) | (ROIs_data == 8))
    
    # load simulated field map in Hz
    b0meas_img = nib.load(meas_fmap_path)
    b0meas_data = b0meas_img.get_fdata()

    # assign variable "x" to indicies corresponding to trachea/lungs
    chi_data[ind_tl] = x

    # compute the field distribution in ppm for the chi_data distribution using the compute_bz function from the susceptibility-to-fieldmap-fft package
    z =  compute_fieldmap.compute_bz(chi_data)
    # convert field distribution from ppm to Hz (assuming 3T)
    z = z*3*42.58 # deltaB0 [Hz] = deltaB0 [ppm] * B0 [T] * 1e-6 * 42.58*e6 [Hz/T]

    return (np.linalg.norm(b0meas_data[ind_sc] - z[ind_sc]))